In [ ]:
import os
import nbimporter

root = os.getcwd().split("survival_analysis")[0]
os.chdir(root + "survival_analysis")

In [ ]:
import pickle

In [ ]:
from nets.cox_nn import CoxNN
from utils.nn_utils import NetTrainer, WeightedBCELoss, SuMoLoss
from nets.monotone_module import MonotonicIncreasingNet
from data_and_preprocessing.dfs_generator import LymphGenerator

# Tests

In [ ]:
df_generator = pickle.load(open("data_and_preprocessing/df_generator_lymph.pickle", "rb" ))
os.mkdir("trained_models/lymph")

# WeightedBCELoss training

In [ ]:
def test_training_with_criterion(criterion):
    net_trainer = NetTrainer(
        df_generator=df_generator,
        patience_factor=1,
        n_training_steps=128,
        model_via_moving_average_on_validation=8,
        criterion=criterion,
        adjust=True,
    )

    monotonic_increasing_net = MonotonicIncreasingNet(latent_sizes=[16]*4)

    model = CoxNN(
        n_input_features=net_trainer.n_input_features,
        monotonic_increasing_net=monotonic_increasing_net,
        t_scaling=net_trainer.horizon,
    )


    model, best_val_score, model_dict = net_trainer.train_and_save(name="Foo", model=model)

    os.remove(f"trained_models/{df_generator.name}/Foo.pickle")

In [ ]:
def test_training_with_WeightedBCELoss():
    criterion = WeightedBCELoss(σ_gaussian_delta=1)
    test_training_with_criterion(criterion)


test_training_with_WeightedBCELoss()

In [ ]:
def test_training_with_SuMoLoss():
    criterion = SuMoLoss()
    test_training_with_criterion(criterion)


test_training_with_SuMoLoss()